In [2]:
# Nombre Jose Angel Graciano Hernandez
# Matricula 100039989
# Asignatura Inteligencia Artificial
# Fecha 4/10/2025

# este proyecto analiza datos usando técnicas o algotimos como KNN, KMeans, PCA entre otas
# se construyeron modelos supervisados para clasificar vinos y comparar algoritmos los cuales son arboles de decision, random forest, KNN y regresión logística.
# random forest mostro mejor desempeño pero KNN capturo patrones locales en los datos de vinos.
# se aplico K-Means para agrupar datos sin etiquetas y se observaron grupos bien definidos.
# se generaron reglas de asociacion para ver cuales productos que se compran juntos el cual lo veo util para promociones o marketing.
# los resultados muestran que combinar modelos y analisis exploratorio ayuda a tomar mejores decisiones.

#Aqui importamos las librerias que vamos a utilizar y tambien al sistema operativo para poder hacer cambios en el 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from mlxtend.frequent_patterns import apriori, association_rules
import os

# En caso de que no tengamos ninguna carpeta o solo tengamos este .py creamos las carpetas
os.makedirs('practica_ia/figuras', exist_ok=True) 
os.makedirs('practica_ia/datos', exist_ok=True)
#Aqui con ese makedirs hacemos las carpetas y el exist_ok va a ser que si las carpetan esta creada no la haga nuevamente


#  parte 1 — arboles de decisión y random forest

# aqui clasificaremos los data set sintentico 
X, y = make_classification(n_samples=800, n_features=6, n_informative=4, n_redundant=0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# aqui en el arbol de desicion ponemos que el random state sea de 42 como lo fue pedido en la asignatura
clf_tree = DecisionTreeClassifier(max_depth=4, random_state=42)
clf_tree.fit(X_train, y_train)

y_pred_tree = clf_tree.predict(X_test)
print("\n--- Árbol de Decisión ---")
print(classification_report(y_test, y_pred_tree))

# aqui podremos visualizar el arbol de decision de manera grafica y este sera guardado en la carpeta de figuras con el nombre de arbol_decision
plt.figure(figsize=(14, 8))
plot_tree(clf_tree, filled=True, feature_names=[f"feat_{i}" for i in range(X.shape[1])])
plt.title("Árbol de Decisión")
plt.savefig('practica_ia/figuras/arbol_decision.png', bbox_inches='tight')
plt.close()

# aqui en este aprendizaje supervisado el cual es random forest tendremos el mismo random state que utilzamos antes
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("\n--- Random Forest ---")
print(classification_report(y_test, y_pred_rf)) #aqui saldra el reporte de la clasificacion del random forest

# Importancia de características
importances = pd.Series(rf.feature_importances_, index=[f"feat_{i}" for i in range(X.shape[1])])
importances.sort_values().plot(kind='barh', title='Importancia de Características (RF)')
plt.tight_layout()
plt.savefig('practica_ia/figuras/importancia_rf.png', bbox_inches='tight')
plt.close()

# parte 2 — comparación KNN vs regresión logística con dataset de vinos

# Carga del dataset de vinos
wine = pd.read_csv('practica_ia/datos/calidad_de_vinos.csv', sep=';', quotechar='"')

# aqui lo que hace es limpiar y verificar columnas
wine = wine.dropna()
print("Columnas detectadas:", wine.columns.tolist())
print("Dimensiones:", wine.shape)

# y verificar por ultimo si la columna quality existe ya que aqui dice la calidad del vino de no haber dicha columna dira que no la hay
if 'quality' not in wine.columns:
    raise ValueError('Se requiere columna quality en dataset de vinos')

# aqui se crear una variable objetiva binaria la cual  1 = buen vino y 0 = regular
y_w = (wine['quality'] >= 7).astype(int)
X_w = wine.drop(columns=['quality'])

# aqui se realiza el escalado
esc = StandardScaler()
X_w_scaled = esc.fit_transform(X_w)

X_train, X_test, y_train, y_test = train_test_split(X_w_scaled, y_w, test_size=0.3, random_state=42)

# aqui inicia le algoritmo el vecino mas cercano o KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

print("\n--- KNN ---")
print(classification_report(y_test, y_pred_knn)) #aqui imprime los resultados del KNN

# aqui inicia el algoritmo regresion logistica
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)

print("\n--- Regresión Logística ---")
print(classification_report(y_test, y_pred_lr)) #imprime los resultados de la regresion logistica

# aqui se haria la comparacion grafica entre KNN y la regresion logistica la cual sera guerdada en la carpeta de figuras con el nombre de comparacion_knn_lr
acc_knn = accuracy_score(y_test, y_pred_knn)
acc_lr = accuracy_score(y_test, y_pred_lr)
plt.bar(['KNN', 'Logistic Regression'], [acc_knn, acc_lr], color=['#66c2a5', '#fc8d62'])
plt.title('Comparación de Precisión')
plt.ylabel('Accuracy')
plt.savefig('practica_ia/figuras/comparacion_knn_lr.png', bbox_inches='tight')
plt.close()

# parte 3 — clustering y PCA con dataset de peces

# cargar el dataset de los peces el cual esta en la carpeta de datos
fish = pd.read_csv('practica_ia/datos/fish.csv')
fish = fish.dropna()

# aqui selecionamos las variables que sean de tipo numero
num_cols = fish.select_dtypes(include=[np.number]).columns

# aqui el escalado
data_scaled = StandardScaler().fit_transform(fish[num_cols])

# aqui iniciamos el algoritmo PCA
pca = PCA(n_components=2)
fish_pca = pca.fit_transform(data_scaled)

# aqui asignamos el random state del KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(fish_pca)

# aqui visualizamos el grafico que el PCA nos dejo el cual sera guardado en la carpeta figuras con el nombre de clustering_fish
plt.figure(figsize=(8, 6))
plt.scatter(fish_pca[:, 0], fish_pca[:, 1], c=clusters, cmap='viridis')
plt.title('Clustering de Peces (PCA)')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.savefig('practica_ia/figuras/clustering_fish.png', bbox_inches='tight')
plt.close()

# parte 4 — reglas de asociación con dataset transaccional sintético

# generamos dataset transaccional sintético
#asignamos el ramdon state o en este caso la random seed a 42 como todas las demas
np.random.seed(42)
items = ['Leche', 'Pan', 'Huevos', 'Queso', 'Mantequilla', 'Jugo', 'Arroz', 'Cereal', 'Pollo', 'Carne', 'Pasta', 'Yogurt']
# creamos varios articulos o items con los nombres que estan arriba
tickets = []
for i in range(200):
    n = np.random.randint(2, 7)
    compra = np.random.choice(items, n, replace=False)
    tickets.append(compra.tolist())

# aqui convertimos la informacion en el formato one hot el cual 
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
oht = te.fit_transform(tickets)
df_trans = pd.DataFrame(oht, columns=te.columns_)

# aqui se aplican reglas de asociacion, por ejemplo si un cliente compra algo la mayoria de los casos comprara algo que atrae otro articulo por medio de este 
# osea ese articulo se asocia a otro
frequent = apriori(df_trans, min_support=0.05, use_colnames=True)
rules = association_rules(frequent, metric='confidence', min_threshold=0.4)

# aqui guarda y muestra las reglas principales de la asociacion y los datos guardado se guardaran en la carpeta datos y se llamara reglas_asociacion 
print("\n--- Principales Reglas de Asociación ---")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

rules.to_csv('practica_ia/datos/reglas_asociacion.csv', index=False)

# conclusiones generales

# KMeans permite detectar patrones en datos sin etiquetas.
# Las reglas de asociación muestran relaciones prácticas entre productos, por ejemplo, huevos y cereal suelen comprarse juntos.


--- Árbol de Decisión ---
              precision    recall  f1-score   support

           0       0.88      0.76      0.82       119
           1       0.80      0.90      0.84       121

    accuracy                           0.83       240
   macro avg       0.84      0.83      0.83       240
weighted avg       0.84      0.83      0.83       240


--- Random Forest ---
              precision    recall  f1-score   support

           0       0.87      0.88      0.88       119
           1       0.88      0.87      0.88       121

    accuracy                           0.88       240
   macro avg       0.88      0.88      0.88       240
weighted avg       0.88      0.88      0.88       240

Columnas detectadas: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
Dimensiones: (1599, 12)

--- KNN ---
              precision    recall  f1-score   support

